In [1]:
import glob
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
def gather_image_from_dir(input_dir):
    image_extensions = ['*.bmp', '*.jpg', '*.png']
    image_list = []
    for image_extension in image_extensions:
        image_list.extend(glob.glob(input_dir + image_extension))
    image_list.sort()
    return image_list

In [3]:
def fit_image_to_screen(image, screen_width=1920, screen_height=1080, scale=0.5):
    """Returns resized, fit to the screen image"""
    height, width = image.shape[:2]
    width_scale = float(screen_width) / float(width)
    height_scale = float(screen_height) / float(height)
    # if image fits to desired screen size, do not resize
    if width_scale > 1.0:
        width_scale = 1.0
    if height_scale > 1.0:
        height_scale = 1.0
    image_scale = height_scale if width_scale > height_scale else width_scale
    image_scale *= scale
    resized_image = cv2.resize(image, (0, 0), fx=image_scale, fy=image_scale)
    return resized_image

In [4]:
def preprocess_image(image):
    # normalize image and make 'tensor-like'
    norm_image = image / 255
    norm_image_norm = np.reshape(norm_image, (1,) + norm_image.shape)
    return norm_image_norm

In [5]:
def resize_and_extrapolate_image_boarders(image, destination_width, destination_height):
    """
    resize to fit image and make boarders
    """
    # resize 'imageROI' to fit into destination image, also keep the aspect ratio
    roi_height, roi_width = image.shape[:2]
    x_aspect_ratio = destination_width / roi_width
    y_aspect_ratio = destination_height / roi_height
    resize_ratio = x_aspect_ratio if x_aspect_ratio < y_aspect_ratio else y_aspect_ratio
    resized_width = int(resize_ratio * float(roi_width))
    resized_height = int(resize_ratio * float(roi_height))
    # prevention from too big width
    if resized_width > destination_width:
        resized_width = destination_width
    if resized_height > destination_height:
        resized_height = destination_height
    delta_w = destination_width - resized_width
    delta_h = destination_height - resized_height
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    resized_roi = cv2.resize(image, (resized_width, resized_height))
    # calculate offset for roi in image (roi is centered)
    """
    resized_roi_height, resized_roi_width = resized_roi.shape[:2]
    x_offset = int((destination_width - resized_roi_width) / 2)
    y_offset = int((destination_height - resized_roi_height) / 2)
    destination_image[y_offset:y_offset+resized_roi_height, x_offset:x_offset+resized_roi_width] = resized_roi
    """
    destination_image = cv2.copyMakeBorder(resized_roi, top, bottom, left, right, cv2.BORDER_REPLICATE, None, [0, 0, 0])
    return destination_image

In [6]:
def create_MobileNetV3Small_model(input_size=(224, 224, 3), weigths_path=None):
    base = tf.keras.applications.MobileNetV3Small(include_top=False,
                                                  weights='imagenet',
                                                  input_shape=input_size)
    base.trainable = True
    model = tf.keras.Sequential([
        base,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    # Draw model graph
    #tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                  optimizer=Adam(lr=1e-3),
                  metrics=['binary_accuracy'])

    if weigths_path != None:
        model.load_weights(weigths_path)
    return model

In [7]:
# check what animal it is from prediction
def cat_or_dog(prediction):
    animal = ''
    if prediction[0][0] >= 0.5:
        animal = 'dog'
    else:
        animal = 'cat'
    return animal

In [8]:

# reset tensorflow values
tf.keras.backend.clear_session()

test_image_dir = 'test_images/'
trained_weights = 'weights/Doggo_or_catto-007-0.0087.hdf5'
model = create_MobileNetV3Small_model(weigths_path=trained_weights)
input_size = (224, 224)
# test directory contains folder with different classes
# collect those subdirectory with different numbers
image_paths = gather_image_from_dir(test_image_dir)
for image_path in image_paths:
    image = cv2.imread(image_path)
    image_preprocessed = resize_and_extrapolate_image_boarders(image, input_size[0], input_size[0])
    image_rb_invert = cv2.cvtColor(image_preprocessed, cv2.COLOR_BGR2RGB)
    image_rb_invert_tensor = preprocess_image(image_rb_invert)
    prediction = model.predict(image_rb_invert_tensor)
    cv2.imshow('original_image', fit_image_to_screen(image))
    cv2.imshow('preprocessed_image', image_preprocessed)
    animal = cat_or_dog(prediction)
    print(animal + ' | model output: ' + str(prediction[0][0]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

cat | model output: 0.0859477
cat | model output: 0.1125396
cat | model output: 0.1290707
cat | model output: 5.72139e-07
cat | model output: 0.017000675
dog | model output: 1.0
dog | model output: 0.999997
dog | model output: 0.99987686
cat | model output: 2.9096198e-10
